In [1]:
from meta_planning.parsers import parse_model, parse_problem, parse_plan
from meta_planning.generator import generate_trajectory
from meta_planning.observations import State, Trajectory
from meta_planning.pddl import Conjunction, Literal, Type, TypedObject, Effect, Truth, NumericConstant, Increase, TypedObject
import copy
import glob
import os
from random import randint, random, shuffle
from itertools import combinations

In [2]:
domain = "grid"
agents = 2

M = parse_model("benchmarks/%s/domain_gen.pddl" % domain)

M.to_file("domain.pddl")

In [3]:
P = parse_problem("benchmarks/%s/problems/problem1.pddl" % domain, M)

In [4]:
# Build Grid

size = 5

objects = []
for i in range(size):
    for j in range(size):
        objects += [TypedObject("loc-x%s-y%s" % (i,j), "tile")]
        

init = []
for i in range(size):
    for j in range(size):
        
        if i > 0:
            init += [Literal("connected", ["loc-x%s-y%s" % (i,j), "loc-x%s-y%s" % (i-1,j)], True)]
        if i < size-1:
            init += [Literal("connected", ["loc-x%s-y%s" % (i,j), "loc-x%s-y%s" % (i+1,j)], True)]
        if j > 0:
            init += [Literal("connected", ["loc-x%s-y%s" % (i,j), "loc-x%s-y%s" % (i,j-1)], True)]
        if j < size-1:
            init += [Literal("connected", ["loc-x%s-y%s" % (i,j), "loc-x%s-y%s" % (i,j+1)], True)]



init += [Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (0,0)], True) for a in range(agents)]

P.objects = objects
P.init = State(init, None)

In [6]:
planner_path = "/home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff"
cmd_args = [planner_path, "-E", "-O", "-o", "domain.pddl", "-f", "real-hyp.pddl", "-s", "solution_plan"]
cmd = " ".join(cmd_args)


goal_tiles = ["loc-x0-y4", "loc-x2-y4", "loc-x4-y4", "loc-x4-y2", "loc-x4-y0"]
# goal_tiles = ["loc-x0-y2", "loc-x0-y0", "loc-x2-y4", "loc-x4-y0", "loc-x4-y2"]
all_hyps = list(combinations(goal_tiles, agents))


num_GRT = 20
num_hyp = 5

for i in range(num_GRT):
    
    mkdir_cmd = "mkdir benchmarks/grid/PGRTG/%s/P%s" % (agents,str(i).zfill(2))
    os.system(mkdir_cmd)
    
    shuffle(all_hyps)    
    hyps = all_hyps[:num_hyp]
    real_hyp = randint(0, num_hyp-1)
    
    f = open("benchmarks/grid/PGRTG/%s/P%s/real-hyp" % (agents, str(i).zfill(2)), "w")
    f.write(str(real_hyp))
    f.close()
    
    for j in range(num_hyp):
        
        new_P = copy.deepcopy(P)

        goal_loc = hyps[j] 
        
        new_P.goal = Conjunction([Literal("at-r%s" % (a+1), [goal_loc[a]], True) for a in range(agents)])

        new_P.to_file("hyp-%s.pddl" % j)

        mv_cmd = "mv hyp-%s.pddl benchmarks/grid/PGRTG/%s/P%s/hyp-%s.pddl" % (j, agents, str(i).zfill(2), j)
        os.system(mv_cmd)
        
        if j == real_hyp:
        
            new_P.to_file("real-hyp.pddl")

            print(cmd)
            os.system(cmd)

            plan = parse_plan("solution_plan")

            print(plan)

            t = generate_trajectory(M, new_P, plan)

            t.to_file("benchmarks/grid/PGRTG/%s/P%s/trajectory" % (agents, str(i).zfill(2)))
        
            


/home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -o domain.pddl -f real-hyp.pddl -s solution_plan
0 : (move-r1 loc-x0-y0 loc-x0-y1)
1 : (move-r1 loc-x0-y1 loc-x0-y2)
2 : (move-r1 loc-x0-y2 loc-x0-y3)
3 : (move-r1 loc-x0-y3 loc-x0-y4)
4 : (move-r2 loc-x0-y0 loc-x0-y1)
5 : (move-r2 loc-x0-y1 loc-x0-y2)
6 : (move-r2 loc-x0-y2 loc-x0-y3)
7 : (move-r2 loc-x0-y3 loc-x0-y4)
8 : (move-r2 loc-x0-y4 loc-x1-y4)
9 : (move-r2 loc-x1-y4 loc-x2-y4)

/home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -o domain.pddl -f real-hyp.pddl -s solution_plan
0 : (move-r1 loc-x0-y0 loc-x0-y1)
1 : (move-r1 loc-x0-y1 loc-x0-y2)
2 : (move-r1 loc-x0-y2 loc-x1-y2)
3 : (move-r1 loc-x1-y2 loc-x2-y2)
4 : (move-r1 loc-x2-y2 loc-x3-y2)
5 : (move-r1 loc-x3-y2 loc-x4-y2)
6 : (move-r2 loc-x0-y0 loc-x1-y0)
7 : (move-r2 loc-x1-y0 loc-x2-y0)
8 : (move-r2 loc-x2-y0 loc-x3-y0)
9 : (move-r2 loc-x3-y0 loc-x4-y0)

/home/dieaigar/PhD/meta-planning/src/meta_planning/u

/home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -o domain.pddl -f real-hyp.pddl -s solution_plan
0 : (move-r1 loc-x0-y0 loc-x0-y1)
1 : (move-r1 loc-x0-y1 loc-x0-y2)
2 : (move-r1 loc-x0-y2 loc-x0-y3)
3 : (move-r1 loc-x0-y3 loc-x0-y4)
4 : (move-r2 loc-x0-y0 loc-x0-y1)
5 : (move-r2 loc-x0-y1 loc-x0-y2)
6 : (move-r2 loc-x0-y2 loc-x0-y3)
7 : (move-r2 loc-x0-y3 loc-x0-y4)
8 : (move-r2 loc-x0-y4 loc-x1-y4)
9 : (move-r2 loc-x1-y4 loc-x2-y4)

/home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -o domain.pddl -f real-hyp.pddl -s solution_plan
0 : (move-r1 loc-x0-y0 loc-x0-y1)
1 : (move-r1 loc-x0-y1 loc-x0-y2)
2 : (move-r1 loc-x0-y2 loc-x0-y3)
3 : (move-r1 loc-x0-y3 loc-x0-y4)
4 : (move-r1 loc-x0-y4 loc-x1-y4)
5 : (move-r1 loc-x1-y4 loc-x2-y4)
6 : (move-r2 loc-x0-y0 loc-x0-y1)
7 : (move-r2 loc-x0-y1 loc-x0-y2)
8 : (move-r2 loc-x0-y2 loc-x1-y2)
9 : (move-r2 loc-x1-y2 loc-x2-y2)
10 : (move-r2 loc-x2-y2 loc-x3-y2)
11 : (move-r2 loc-